#  Hospital Data ETL Demonstration

This notebook demonstrates **Python-based ETL (Extract, Transform, Load)** steps on a hospital dataset.  
It is intended to showcase **data engineering skills** using Python, while the final **visualizations and reporting** will be built in **Power BI**.

---

## 1. Setup & Libraries

In [ ]:
import pandas as pd
import os

## 2. Load Hospital Data Tables

In [ ]:
# Define folder path where CSV files are located
folder_path = r'C:\Users\nawaf\Downloads\HospetalDB'  # change if needed

# List CSV files in folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Load CSV files into a dictionary
dataframes = {f.split('.csv')[0].strip().lower(): pd.read_csv(os.path.join(folder_path, f))
              for f in csv_files}

# Assign variables
appointments = dataframes.get('appointments')
patients = dataframes.get('patients')
doctors = dataframes.get('doctors')
treatments = dataframes.get('treatments')
beeling = dataframes.get('billing')


## 3. Data Overview

In [ ]:

# Display shapes of all tables
print("Patients:", patients.shape)
print("Appointments:", appointments.shape)
print("Doctors:", doctors.shape)
print("Treatments:", treatments.shape)
print("Billing:", billing.shape)

# Quick info and missing values check
patients.info()
patients.isnull().sum()
    

## 4. General + Healthcare-Specific Transformations

In [126]:
# 1. rename columns for consistency
patients = patients.rename(columns={"Name": "patient_name", "Age": "patient_age"})
print(patients.head())

  patient_id first_name last_name gender date_of_birth  contact_number  \
0       P001      David  Williams      F    1955-06-04      6939585183   
1       P002      Emily     Smith      F    1984-10-12      8228188767   
2       P003      Laura     Jones      M    1977-08-21      8397029847   
3       P004    Michael   Johnson      F    1981-02-20      9019443432   
4       P005      David    Wilson      M    1960-06-23      7734463155   

        address registration_date insurance_provider insurance_number  \
0   789 Pine Rd        2022-06-23       WellnessCorp        INS840674   
1  321 Maple Dr        2022-01-15        PulseSecure        INS354079   
2  321 Maple Dr        2022-02-07        PulseSecure        INS650929   
3    123 Elm St        2021-03-02        HealthIndia        INS789944   
4    123 Elm St        2021-09-29       MedCare Plus        INS788105   

                      email  
0   david.williams@mail.com  
1      emily.smith@mail.com  
2      laura.jones@mail.co

In [169]:
# 2. standardize text formatting
patients['first_name'] = patients['first_name'].str.strip().str.title()
print(patients.head())

  patient_id first_name last_name  gender date_of_birth  contact_number  \
0       P001      David  Williams  Female    1955-06-04      6939585183   
1       P002      Emily     Smith  Female    1984-10-12      8228188767   
2       P003      Laura     Jones    Male    1977-08-21      8397029847   
3       P004    Michael   Johnson  Female    1981-02-20      9019443432   
4       P005      David    Wilson    Male    1960-06-23      7734463155   

        address registration_date insurance_provider insurance_number  \
0   789 Pine Rd        2022-06-23       WellnessCorp        INS840674   
1  321 Maple Dr        2022-01-15        PulseSecure        INS354079   
2  321 Maple Dr        2022-02-07        PulseSecure        INS650929   
3    123 Elm St        2021-03-02        HealthIndia        INS789944   
4    123 Elm St        2021-09-29       MedCare Plus        INS788105   

                      email  
0   david.williams@mail.com  
1      emily.smith@mail.com  
2      laura.jones@m

In [130]:
# 3. convert date columns to datetime
appointments['appointment_date'] = pd.to_datetime(appointments['appointment_date'])
print(appointments.head())

  appointment_id patient_id doctor_id appointment_date appointment_time  \
0           A001       P034      D009       2023-08-09         15:15:00   
1           A002       P032      D004       2023-06-09         14:30:00   
2           A003       P048      D004       2023-06-28          8:00:00   
3           A004       P025      D006       2023-09-01          9:15:00   
4           A005       P040      D003       2023-07-06         12:45:00   

  reason_for_visit     status  
0          Therapy  Scheduled  
1          Therapy    No-show  
2     Consultation  Cancelled  
3     Consultation  Cancelled  
4        Emergency    No-show  


In [185]:
# 4. calculate patient age from date_of_birth and handle missing values
patients['date_of_birth'] = pd.to_datetime(patients['date_of_birth'], errors='coerce')  # convert to datetime
today = pd.Timestamp('today')
patients['patient_age'] = (today - patients['date_of_birth']).dt.days // 365  # age in years
patients['patient_age'] = patients['patient_age'].fillna(patients['patient_age'].median())  # fill missing
print(patients.head())


  patient_id first_name last_name  gender date_of_birth  contact_number  \
0       P001      David  Williams  Female    1955-06-04      6939585183   
1       P002      Emily     Smith  Female    1984-10-12      8228188767   
2       P003      Laura     Jones    Male    1977-08-21      8397029847   
3       P004    Michael   Johnson  Female    1981-02-20      9019443432   
4       P005      David    Wilson    Male    1960-06-23      7734463155   

        address registration_date insurance_provider insurance_number  \
0   789 Pine Rd        2022-06-23       WellnessCorp        INS840674   
1  321 Maple Dr        2022-01-15        PulseSecure        INS354079   
2  321 Maple Dr        2022-02-07        PulseSecure        INS650929   
3    123 Elm St        2021-03-02        HealthIndia        INS789944   
4    123 Elm St        2021-09-29       MedCare Plus        INS788105   

                      email  patient_age  
0   david.williams@mail.com           70  
1      emily.smith@mail.

In [187]:
# 5. categorize age groups
patients['age_group'] = pd.cut(
    patients['patient_age'],
    bins=[0, 12, 19, 35, 60, 100],
    labels=['Child', 'Teen', 'Young Adult', 'Adult', 'Senior']
)
print(patients.head())

  patient_id first_name last_name  gender date_of_birth  contact_number  \
0       P001      David  Williams  Female    1955-06-04      6939585183   
1       P002      Emily     Smith  Female    1984-10-12      8228188767   
2       P003      Laura     Jones    Male    1977-08-21      8397029847   
3       P004    Michael   Johnson  Female    1981-02-20      9019443432   
4       P005      David    Wilson    Male    1960-06-23      7734463155   

        address registration_date insurance_provider insurance_number  \
0   789 Pine Rd        2022-06-23       WellnessCorp        INS840674   
1  321 Maple Dr        2022-01-15        PulseSecure        INS354079   
2  321 Maple Dr        2022-02-07        PulseSecure        INS650929   
3    123 Elm St        2021-03-02        HealthIndia        INS789944   
4    123 Elm St        2021-09-29       MedCare Plus        INS788105   

                      email  patient_age age_group  
0   david.williams@mail.com           70    Senior  
1   

In [136]:
# 6. map gender values
patients['gender'] = patients['gender'].replace({'M': 'Male', 'F': 'Female'})
print(patients.head())

  patient_id first_name last_name  gender date_of_birth  contact_number  \
0       P001      David  Williams  Female    1955-06-04      6939585183   
1       P002      Emily     Smith  Female    1984-10-12      8228188767   
2       P003      Laura     Jones    Male    1977-08-21      8397029847   
3       P004    Michael   Johnson  Female    1981-02-20      9019443432   
4       P005      David    Wilson    Male    1960-06-23      7734463155   

        address registration_date insurance_provider insurance_number  \
0   789 Pine Rd        2022-06-23       WellnessCorp        INS840674   
1  321 Maple Dr        2022-01-15        PulseSecure        INS354079   
2  321 Maple Dr        2022-02-07        PulseSecure        INS650929   
3    123 Elm St        2021-03-02        HealthIndia        INS789944   
4    123 Elm St        2021-09-29       MedCare Plus        INS788105   

                      email  
0   david.williams@mail.com  
1      emily.smith@mail.com  
2      laura.jones@m

In [193]:
# 7. set default treatment duration since end date is not available
treatments['treatment_duration'] = 1
print(treatments.head())


  treatment_id appointment_id treatment_type         description     cost  \
0         T001           A001   Chemotherapy     Basic screening  3941.97   
1         T002           A002            MRI   Advanced protocol  4158.44   
2         T003           A003            MRI  Standard procedure  3731.55   
3         T004           A004            MRI     Basic screening  4799.86   
4         T005           A005            ECG  Standard procedure   582.05   

  treatment_date  treatment_duration  
0     2023-08-09                   1  
1     2023-06-09                   1  
2     2023-06-28                   1  
3     2023-09-01                   1  
4     2023-07-06                   1  


In [140]:
# 8. merge appointments with patients
appt_patients = appointments.merge(patients, on="patient_id", how="left")
print(appt_patients.head())

  appointment_id patient_id doctor_id appointment_date appointment_time  \
0           A001       P034      D009       2023-08-09         15:15:00   
1           A002       P032      D004       2023-06-09         14:30:00   
2           A003       P048      D004       2023-06-28          8:00:00   
3           A004       P025      D006       2023-09-01          9:15:00   
4           A005       P040      D003       2023-07-06         12:45:00   

  reason_for_visit     status first_name last_name  gender date_of_birth  \
0          Therapy  Scheduled       Alex     Smith  Female    1950-01-26   
1          Therapy    No-show       Alex     Moore    Male    1981-01-08   
2     Consultation  Cancelled      Emily    Miller    Male    1983-03-24   
3     Consultation  Cancelled     Robert    Wilson    Male    1966-08-14   
4        Emergency    No-show      Emily  Williams    Male    1972-05-30   

   contact_number       address registration_date insurance_provider  \
0      8374657733  3

In [142]:
# 9. merge appointments with doctors
appt_doctors = appointments.merge(doctors, on="doctor_id", how="left")
print(appt_doctors.head())

  appointment_id patient_id doctor_id appointment_date appointment_time  \
0           A001       P034      D009       2023-08-09         15:15:00   
1           A002       P032      D004       2023-06-09         14:30:00   
2           A003       P048      D004       2023-06-28          8:00:00   
3           A004       P025      D006       2023-09-01          9:15:00   
4           A005       P040      D003       2023-07-06         12:45:00   

  reason_for_visit     status first_name last_name specialization  \
0          Therapy  Scheduled      Sarah     Smith     Pediatrics   
1          Therapy    No-show      David     Jones     Pediatrics   
2     Consultation  Cancelled      David     Jones     Pediatrics   
3     Consultation  Cancelled       Alex     Davis     Pediatrics   
4        Emergency    No-show       Jane     Smith     Pediatrics   

   phone_number  years_experience   hospital_branch  \
0    7387087517                26  Central Hospital   
1    6594221991         

In [144]:
# 10. merge billing with patients
billing_patients = peeling.merge(patients, on="patient_id", how="left")
print(billing_patients.head())

  bill_id patient_id treatment_id   bill_date   amount payment_method  \
0    B001       P034         T001  2023-08-09  3941.97      Insurance   
1    B002       P032         T002  2023-06-09  4158.44      Insurance   
2    B003       P048         T003  2023-06-28  3731.55      Insurance   
3    B004       P025         T004  2023-09-01  4799.86      Insurance   
4    B005       P040         T005  2023-07-06   582.05    Credit Card   

  payment_status first_name last_name  gender date_of_birth  contact_number  \
0        Pending       Alex     Smith  Female    1950-01-26      8374657733   
1           Paid       Alex     Moore    Male    1981-01-08      8102183595   
2           Paid      Emily    Miller    Male    1983-03-24      8720989381   
3         Failed     Robert    Wilson    Male    1966-08-14      7482069727   
4        Pending      Emily  Williams    Male    1972-05-30      7587653815   

        address registration_date insurance_provider insurance_number  \
0  321 Maple 

In [146]:
# 11. total bill per patient
billing_summary = peeling.groupby('patient_id')['amount'].sum().reset_index(name='total_bill')
print(billing_summary.head())

  patient_id  total_bill
0       P001     7719.07
1       P002     5968.74
2       P003     7936.88
3       P004     5362.51
4       P005    18609.91


In [148]:
# 12. find frequent visitors
frequent_visitors = appointments['patient_id'].value_counts().reset_index()
frequent_visitors.columns = ['patient_id', 'num_visits']
print(frequent_visitors.head())

  patient_id  num_visits
0       P012          10
1       P005           8
2       P016           7
3       P036           7
4       P029           7


In [150]:
# 13. patients without appointments
no_appt = patients[~patients['patient_id'].isin(appointments['patient_id'])]
print(no_appt.head())

   patient_id first_name last_name gender date_of_birth  contact_number  \
5        P006      Linda     Jones   Male    1963-06-16      7561777264   
14       P015      Sarah   Johnson   Male    1964-05-11      6636028516   

         address registration_date insurance_provider insurance_number  \
5   321 Maple Dr        2022-10-02        HealthIndia        INS613758   
14  321 Maple Dr        2021-09-25       WellnessCorp        INS922209   

                     email  
5     linda.jones@mail.com  
14  sarah.johnson@mail.com  


In [152]:
# 14. remove duplicate patients
patients = patients.drop_duplicates(subset=['patient_id'])
print(patients.head())

  patient_id first_name last_name  gender date_of_birth  contact_number  \
0       P001      David  Williams  Female    1955-06-04      6939585183   
1       P002      Emily     Smith  Female    1984-10-12      8228188767   
2       P003      Laura     Jones    Male    1977-08-21      8397029847   
3       P004    Michael   Johnson  Female    1981-02-20      9019443432   
4       P005      David    Wilson    Male    1960-06-23      7734463155   

        address registration_date insurance_provider insurance_number  \
0   789 Pine Rd        2022-06-23       WellnessCorp        INS840674   
1  321 Maple Dr        2022-01-15        PulseSecure        INS354079   
2  321 Maple Dr        2022-02-07        PulseSecure        INS650929   
3    123 Elm St        2021-03-02        HealthIndia        INS789944   
4    123 Elm St        2021-09-29       MedCare Plus        INS788105   

                      email  
0   david.williams@mail.com  
1      emily.smith@mail.com  
2      laura.jones@m

In [207]:
# create due_date: 30 days after bill_date
billing['due_date'] = pd.to_datetime(billing['bill_date']) + pd.Timedelta(days=30)

# create payment_date: assume paid on bill_date if 'Paid', else NaT
billing['payment_date'] = pd.to_datetime(
    billing['bill_date'].where(billing['payment_status'] == 'Paid', pd.NaT)
)

# flag late payments
billing['is_late'] = billing['payment_date'] > billing['due_date']

print(billing.head())

  bill_id patient_id treatment_id   bill_date   amount payment_method  \
0    B001       P034         T001  2023-08-09  3941.97      Insurance   
1    B002       P032         T002  2023-06-09  4158.44      Insurance   
2    B003       P048         T003  2023-06-28  3731.55      Insurance   
3    B004       P025         T004  2023-09-01  4799.86      Insurance   
4    B005       P040         T005  2023-07-06   582.05    Credit Card   

  payment_status   due_date payment_date  is_late  
0        Pending 2023-09-08          NaT    False  
1           Paid 2023-07-09   2023-06-09    False  
2           Paid 2023-07-28   2023-06-28    False  
3         Failed 2023-10-01          NaT    False  
4        Pending 2023-08-05          NaT    False  


In [217]:
# 18. top 5 doctors by patient count
top_doctors = appointments.groupby('doctor_id')['patient_id'].nunique().nlargest(5).reset_index()
print(top_doctors.head())

  doctor_id  patient_id
0      D005          23
1      D001          20
2      D006          19
3      D003          18
4      D002          17


In [219]:
# 19. average billing per treatment type
billing_treatment = peeling.merge(treatments, on="treatment_id", how="left")
avg_bill = billing_treatment.groupby('treatment_type')['amount'].mean().reset_index()
print(avg_bill.head())

  treatment_type       amount
0   Chemotherapy  2629.707755
1            ECG  2532.216842
2            MRI  3224.948889
3  Physiotherapy  2761.613889
4          X-Ray  2698.870000


In [221]:
# 20. calculate patient lifetime value (LTV)
ltv = peeling.groupby('patient_id')['amount'].sum().reset_index(name='lifetime_value')
print(ltv.head())

  patient_id  lifetime_value
0       P001         7719.07
1       P002         5968.74
2       P003         7936.88
3       P004         5362.51
4       P005        18609.91


## 5. Export Cleaned Data for Power BI

In [ ]:

# Save cleaned tables for Power BI (CSV format)
export_path = os.path.join(folder_path, "cleaned")

os.makedirs(export_path, exist_ok=True)

patients.to_csv(os.path.join(export_path, "patients_clean.csv"), index=False)
appointments.to_csv(os.path.join(export_path, "appointments_clean.csv"), index=False)
doctors.to_csv(os.path.join(export_path, "doctors_clean.csv"), index=False)
treatments.to_csv(os.path.join(export_path, "treatments_clean.csv"), index=False)
billing.to_csv(os.path.join(export_path, "billing_clean.csv"), index=False)

print("✅ Cleaned data exported successfully for Power BI.")
    


## 6. Closing Note

The dataset is relatively **small and already structured**, which makes **Power BI the ideal tool for interactive dashboards and reporting**.  

This notebook demonstrates how Python can be used for **ETL and data preparation**, complementing Power BI for **business intelligence delivery**.  
Together, they provide an end-to-end workflow: **Python for ETL + Power BI for visualization**.
    

In [211]:
print(appointments.columns)


Index(['appointment_id', 'patient_id', 'doctor_id', 'appointment_date',
       'appointment_time', 'reason_for_visit', 'status'],
      dtype='object')
